In [44]:
# import the required libraries

import os
import re
import csv
import pandas as pd
import numpy as np


In [45]:
# Set variables for the project (i.e. the input location of the file to be processed and the output location) )

folderlink = '..//data//'
input_folder = 'input//'
input_file = os.path.join(folderlink+input_folder, 'boekzaallijst_27072023.csv')
folder_output = 'output//'
output_csv = folderlink+folder_output+'clerus_boekzaal.csv'
input_file_parent = folderlink+folder_output+"parent_data.csv"
input_file_minister = folderlink+folder_output+"minister_info.csv"

# Panda settings for showing data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [46]:
years_to_integers = {'Jaar (Begin) Rol': pd.Int64Dtype(),'Jaar Beroepen': pd.Int64Dtype(), }
boekzaallijst = pd.read_csv(input_file, sep=';', dtype=years_to_integers, encoding='utf-8')

In [55]:
boekzaallijst.head()

,Nr_BZ,Achternaam_BZ,Tussenvoegsel_BZ,Voornaam_BZ,Voorletter_BZ,Variate_Achternaam_BZ,Variate_Voornaam_BZ,Variatie_Voorletter_BZ,Family name,Toevoeging,Classis,Rol,Rol plaats,Jaar (Begin) Rol,Periode (eind) Rol,Jaar 2 en verder rol,Periode 2 (en verder) rol,Jaar (eind) rol,Periode (eind) rol,Jaar Beroepen,Periode (Beroepen),Rol.1,Bestemming,Toevoeging plaats bestemming,Aantekening bij bestemming,Overleden,Jaar overlijden,Extra functie,Locatie xtra functie,Jaar begin xtra functie,Periode xtra functie,Unnamed: 31,Periode (einde),Aantekening xtra functie,Achternaam_BZ_3
0,1,Aaken,van,NaN,J.,NaN,NaN,J.H.,NaN,NaN,13,Proponent,NaN,1815,AB,NaN,NaN,NaN,NaN,1816,A,Predikant,St. Michiels Gestel en Gemunde,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aaken
1,2,Aalburg,van,Joannes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,1743,B,Predikant,Oudkarspel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aalburg
2,3,Aardenburg,van,Buwama,C.,NaN,NaN,NaN,NaN,NaN,52,Proponent,NaN,1716,A(B),NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aardenburg
3,4,Aardse,NaN,Albertus,NaN,NaN,NaN,NaN,NaN,NaN,44,Proponent,NaN,1757,B,1758,AB,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aardse
4,5,Aartsen,NaN,Albertus,NaN,NaN,NaN,NaN,NaN,NaN,40,Proponent,NaN,1766,AB,1767,AB,NaN,NaN,1776,B,Predikant,Randwijk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aartsen


In [47]:
#boekzaallijst['Achternaam_BZ_3']= boekzaallijst['Achternaam_BZ'].str[:10]

In [65]:

boekzaallijst['boekzaallink'] = boekzaallijst['Achternaam_BZ'].astype(str) + '_' + boekzaallijst['Bestemming'].astype(str).str.replace(' ', '').str[:6]

In [51]:
years_to_integers = {'minister_year': pd.Int64Dtype() }
minister = pd.read_csv(input_file_minister, sep=';', dtype=years_to_integers, encoding='utf-8')

In [52]:
minister.head()

,clerus_id,minister,join_name,original_input,minister_year,accu_year_minster
0,1,Oudkarspel,Aalburg Johannes van,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",1743,NaN
1,4,Aardenburg,Aalst Wilhelmus,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>",1695,NaN
2,2,Kalslagen ber,Aalst Cornelius van,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>",1717,NaN
3,3,Vuren en Dalem,Aalst Gerardus van,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",1704,NaN
4,3,Sommelsdijk,Aalst Gerardus van,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",1706,NaN


In [71]:
minister['join_name_1'] = minister['join_name'].str.split().str[0]
minister['boekzaallink'] = minister['join_name_1'].astype(str) + '_' + minister['minister'].astype(str).str.replace(' ', '').str[:6]

In [72]:
minister.head()

,clerus_id,minister,join_name,original_input,minister_year,accu_year_minster,boekzaallink,join_name_1,boekzaallink2
0,1,Oudkarspel,Aalburg Johannes van,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",1743,NaN,Aalburg_Oudkar,Aalburg,Aalburg_Oudkar
1,4,Aardenburg,Aalst Wilhelmus,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>",1695,NaN,Aalst_Aarden,Aalst,Aalst_Aarden
2,2,Kalslagen ber,Aalst Cornelius van,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>",1717,NaN,Aalst_Kalsla,Aalst,Aalst_Kalsla
3,3,Vuren en Dalem,Aalst Gerardus van,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",1704,NaN,Aalst_Vurene,Aalst,Aalst_Vurene
4,3,Sommelsdijk,Aalst Gerardus van,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",1706,NaN,Aalst_Sommel,Aalst,Aalst_Sommel


In [77]:
boekzaal_minister = pd.merge(boekzaallijst, minister, left_on='boekzaallink', right_on='boekzaallink', how='left')

In [78]:
boekzaal_minister.head()

,Nr_BZ,Achternaam_BZ,Tussenvoegsel_BZ,Voornaam_BZ,Voorletter_BZ,Variate_Achternaam_BZ,Variate_Voornaam_BZ,Variatie_Voorletter_BZ,Family name,Toevoeging,Classis,Rol,Rol plaats,Jaar (Begin) Rol,Periode (eind) Rol,Jaar 2 en verder rol,Periode 2 (en verder) rol,Jaar (eind) rol,Periode (eind) rol,Jaar Beroepen,Periode (Beroepen),Rol.1,Bestemming,Toevoeging plaats bestemming,Aantekening bij bestemming,Overleden,Jaar overlijden,Extra functie,Locatie xtra functie,Jaar begin xtra functie,Periode xtra functie,Unnamed: 31,Periode (einde),Aantekening xtra functie,Achternaam_BZ_3,boekzaallink,clerus_id,minister,join_name,original_input,minister_year,accu_year_minster,join_name_1,boekzaallink2
0,1,Aaken,van,NaN,J.,NaN,NaN,J.H.,NaN,NaN,13,Proponent,NaN,1815,AB,NaN,NaN,NaN,NaN,1816,A,Predikant,St. Michiels Gestel en Gemunde,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aaken,Aaken_St.Mic,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
1,2,Aalburg,van,Joannes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,1743,B,Predikant,Oudkarspel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aalburg,Aalburg_Oudkar,1.0,Oudkarspel,Aalburg Johannes van,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",1743,NaN,Aalburg,Aalburg_Oudkar
2,3,Aardenburg,van,Buwama,C.,NaN,NaN,NaN,NaN,NaN,52,Proponent,NaN,1716,A(B),NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aardenburg,Aardenburg_nan,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
3,4,Aardse,NaN,Albertus,NaN,NaN,NaN,NaN,NaN,NaN,44,Proponent,NaN,1757,B,1758,AB,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aardse,Aardse_nan,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
4,5,Aartsen,NaN,Albertus,NaN,NaN,NaN,NaN,NaN,NaN,40,Proponent,NaN,1766,AB,1767,AB,NaN,NaN,1776,B,Predikant,Randwijk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aartsen,Aartsen_Randwi,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN


In [79]:
boekzaal_minister.describe()

,Nr_BZ,Jaar (Begin) Rol,Jaar (eind) rol,Jaar Beroepen,Jaar overlijden,Unnamed: 31,clerus_id,minister_year,accu_year_minster
count,4826.000000,3781.000000,22.000000,3878.000000,2.000000,2.000000,1601.000000,1522.000000,0.0
mean,2346.438458,1769.180376,1781.636364,1774.663486,1761.000000,1768.500000,5775.830106,1775.891590,NaN
std,1356.451109,23.252746,21.115432,22.746270,7.071068,24.748737,3049.300268,25.502409,NaN
min,1.000000,1646.000000,1751.000000,1736.000000,1756.000000,1751.000000,1.000000,1609.000000,NaN
25%,1175.250000,1750.000000,1764.000000,1755.000000,1758.500000,1759.750000,3545.000000,1758.000000,NaN
50%,2344.500000,1766.000000,1781.500000,1775.000000,1761.000000,1768.500000,5770.000000,1779.000000,NaN
75%,3520.750000,1786.000000,1797.750000,1792.000000,1763.500000,1777.250000,8276.000000,1795.750000,NaN
max,4705.000000,1865.000000,1816.000000,1882.000000,1766.000000,1786.000000,11213.000000,1834.000000,NaN


In [80]:
boekzaal_minister.to_csv(folderlink+folder_output+'boekzaal_minister2.csv', sep=';', encoding='utf-8', index=False)